In [ ]:
from models import NvidiaModel, RMSELoss
from torch.utils.data import Dataset, DataLoader
import utils as ut
import torch
import train_nvidia_model as tnm
import os 
import numpy as np
import random 

In [ ]:
seed = 1729 
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
class FrameDataset(Dataset):
    def __init__(self, frames_dir, angles_file, start_idx, end_idx):
        self.frames = ut.load_frames(frames_dir, start_idx, end_idx)
        self.angles = torch.tensor(ut.read_angles(angles_file)[start_idx:end_idx])

    def __getitem__(self, idx):
        return (self.frames[idx], self.angles[idx])

    def __len__(self):
        return len(self.angles)

In [ ]:
data_dir = "data_h3_w3"
videos = [0, 1, 2, 3, 4]
train_dir = os.path.join(data_dir, "train")
train_dirs = [os.path.join(train_dir, str(video)) for video in videos]
angles_files = [os.path.join(td, "angles.txt") for td in train_dirs]

In [ ]:
train_datasets = [FrameDataset(frame_dir, angles_file, 0, 2) for frame_dir, angles_file in zip(train_dirs, angles_files)]

In [ ]:
batch_size = 2
def seed_worker(worker_id):
    # worker_seed = torch.initial_seed() % 2**32
    worker_seed = seed
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(seed)

train_dataloader = [DataLoader(train_ds, batch_size, worker_init_fn=seed_worker, generator=g) for train_ds in train_datasets]

In [ ]:
model = NvidiaModel()
opt = torch.optim.AdamW(model.parameters(), lr=3e-04)
loss = RMSELoss()
history = tnm.History("test_1", videos, model, opt, loss, None, batch_size)
epochs = 15
tnm.fit(epochs, history, train_dataloader, None, dev, verbose=True)
history.save_training_info()

In [ ]:
history.test_model()